In [1]:
import pandas as pd 
import random
import os
from datetime import datetime
import hashlib
from sqlalchemy import create_engine 
from snowflake.sqlalchemy import URL

In [91]:
SNOW_USER = "gabrielgarcia14"
SNOW_PASS = "ZzZZzZzZZZ"

# ENGINE FOR READING DATA:
test_eng = create_engine(URL( 
    user      = SNOW_USER,
    password  = SNOW_PASS, 
    account   = "epb29041.us-east-1",
    warehouse = 'WH_GABO_PROD',
    database  = 'ALEXPROJECT',
    schema    = 'PUBLIC',
    role      = 'ACCOUNTADMIN',
    numpy     =  True,
)) 

def read_snow(snow_engine,snow_query="SELECT current_version()"):
    con = snow_engine.connect() 
    df1 = pd.read_sql_query(snow_query,snow_engine) 
    df1.columns = [a.upper().strip() for a in df1.columns]
    con.close() 
    return df1

def write_snow(snow_engine,data_frame,table_name="test_table",method='append'):
    con = snow_engine.connect() 
    table_name = table_name.lower() 
    if method[0].lower()=='a': method='append'
    if method[0].lower()=='r': method='replace'
    data_frame.to_sql(
        name = table_name, 
        con = snow_engine,  
        if_exists = method,  
        index = False) 
    con.close() 


In [3]:
dict_leagues = {
    'SPAIN' : 'esp',
    'ENGLAND' : 'eng',
    'ITALY' : 'ita',
    'GERMANY' : 'ger',
    'FRANCE' : 'fra',
    'PORTUGAL' : 'por',
    'NETHERLANDS' : 'ned',
}

In [2]:
countries = ['SPAIN', 'ENGLAND', 'ITALY', 'GERMANY', 'FRANCE', 'PORTUGAL', 'NETHERLANDS', 'COLOMBIA']
codes = ['esp', 'eng', 'ita', 'ger', 'fra', 'por', 'ned', 'col']


In [3]:
for country, code in zip(countries, codes):
    print(country,  code)

SPAIN esp
ENGLAND eng
ITALY ita
GERMANY ger
FRANCE fra
PORTUGAL por
NETHERLANDS ned
COLOMBIA col


In [30]:
 # Generate the url
url = f'https://www.espn.com.co/futbol/posiciones/_/liga/fra.1'
# Create a dataframe with the tables from the URL
df = pd.read_html(url)

In [33]:
countries

['SPAIN',
 'ENGLAND',
 'ITALY',
 'GERMANY',
 'FRANCE',
 'PORTUGAL',
 'NETHERLANDS',
 'COLOMBIA']

In [48]:
list_df = []

for country, code in zip(countries, codes):
    # Generate the url
    url = f'https://www.espn.com.co/futbol/posiciones/_/liga/{code}.1'
    # Create a dataframe with the tables from the URL
    df = pd.read_html(url)
    # df is a list with two dataframes. One with the names of the footbal teams, and the another with the statistics from the team
    # Rename the column of the first df
    df[0].rename(columns={df[0].columns[0] : 'FOOTBALL_TEAM'}, inplace=True)
    # join both dfs into a new one
    df = pd.concat([df[0], df[1]], axis = 1)
    # assign the country
    df['COUNTRY'] = country
    list_df.append(df)

In [54]:
df_leagues = pd.concat(list_df, axis = 0)

In [57]:
df_leagues

,FOOTBALL_TEAM,J,G,E,P,GF,GC,DIF,PTS,COUNTRY
0,1BARBarcelona,24,20,2,2,46,8,38,62,SPAIN
1,2MADReal Madrid,24,16,5,3,47,18,29,53,SPAIN
2,3ATLAtlético Madrid,24,13,6,5,38,19,19,45,SPAIN
3,4SOCReal Sociedad,24,13,5,6,32,23,9,44,SPAIN
4,5BETReal Betis,24,12,5,7,32,25,7,41,SPAIN
...,...,...,...,...,...,...,...,...,...,...
15,16TOLDeportes Tolima,6,1,3,2,7,9,-2,6,COLOMBIA
16,17SFEIndependiente Santa Fe,6,1,3,2,6,8,-2,6,COLOMBIA
17,18DCIDeportivo Cali,6,1,3,2,4,6,-2,6,COLOMBIA
18,19APAAlianza Petrolera,5,1,2,2,3,4,-1,5,COLOMBIA


In [58]:
# Clean the football team column. 
df_leagues['FOOTBALL_TEAM'] = df_leagues['FOOTBALL_TEAM'].apply(lambda x: x[5:] if x[:2].isnumeric() == True else x[4:])

In [59]:
df_leagues

,FOOTBALL_TEAM,J,G,E,P,GF,GC,DIF,PTS,COUNTRY
0,Barcelona,24,20,2,2,46,8,38,62,SPAIN
1,Real Madrid,24,16,5,3,47,18,29,53,SPAIN
2,Atlético Madrid,24,13,6,5,38,19,19,45,SPAIN
3,Real Sociedad,24,13,5,6,32,23,9,44,SPAIN
4,Real Betis,24,12,5,7,32,25,7,41,SPAIN
...,...,...,...,...,...,...,...,...,...,...
15,Deportes Tolima,6,1,3,2,7,9,-2,6,COLOMBIA
16,Independiente Santa Fe,6,1,3,2,6,8,-2,6,COLOMBIA
17,Deportivo Cali,6,1,3,2,4,6,-2,6,COLOMBIA
18,Alianza Petrolera,5,1,2,2,3,4,-1,5,COLOMBIA


In [76]:
def id_generator(team):

    return hashlib.md5(f"{team}".encode()).hexdigest()[:10]

In [77]:
df_leagues.apply(lambda row: id_generator(row['COUNTRY'] + row['FOOTBALL_TEAM']), axis = 1)

0     5b56ddfddc
1     d41719b640
2     bc3b1e972a
3     fa11a6efaf
4     498c0722bf
         ...    
15    5301f61042
16    c8b4ab6fff
17    8cd73c5d68
18    46bfec9f1d
19    0e030f06ef
Length: 154, dtype: object

In [79]:
df_leagues['ID'] = '533235'

In [85]:
df_leagues.insert(loc = 0,
          column = 'ID',
          value = df_leagues.apply(lambda row: id_generator(row['COUNTRY'] + row['FOOTBALL_TEAM']), axis = 1)
          )

In [86]:
df_leagues

,ID,FOOTBALL_TEAM,J,G,E,P,GF,GC,DIF,PTS,COUNTRY
0,5b56ddfddc,Barcelona,24,20,2,2,46,8,38,62,SPAIN
1,d41719b640,Real Madrid,24,16,5,3,47,18,29,53,SPAIN
2,bc3b1e972a,Atlético Madrid,24,13,6,5,38,19,19,45,SPAIN
3,fa11a6efaf,Real Sociedad,24,13,5,6,32,23,9,44,SPAIN
4,498c0722bf,Real Betis,24,12,5,7,32,25,7,41,SPAIN
...,...,...,...,...,...,...,...,...,...,...,...
15,5301f61042,Deportes Tolima,6,1,3,2,7,9,-2,6,COLOMBIA
16,c8b4ab6fff,Independiente Santa Fe,6,1,3,2,6,8,-2,6,COLOMBIA
17,8cd73c5d68,Deportivo Cali,6,1,3,2,4,6,-2,6,COLOMBIA
18,46bfec9f1d,Alianza Petrolera,5,1,2,2,3,4,-1,5,COLOMBIA


In [87]:
df_leagues['ID'].value_counts()

5b56ddfddc    1
8b4dbc0081    1
26d20ebf62    1
2153db46e7    1
bd073550cf    1
             ..
60195f0908    1
17715441ea    1
5165f0849b    1
043210ac1b    1
0e030f06ef    1
Name: ID, Length: 154, dtype: int64

In [88]:
df_leagues['CREATED_TIME'] = datetime.now().strftime('%Y-%m-%d')

In [89]:
df_leagues

,ID,FOOTBALL_TEAM,J,G,E,P,GF,GC,DIF,PTS,COUNTRY,CREATED_TIME
0,5b56ddfddc,Barcelona,24,20,2,2,46,8,38,62,SPAIN,2023-03-05
1,d41719b640,Real Madrid,24,16,5,3,47,18,29,53,SPAIN,2023-03-05
2,bc3b1e972a,Atlético Madrid,24,13,6,5,38,19,19,45,SPAIN,2023-03-05
3,fa11a6efaf,Real Sociedad,24,13,5,6,32,23,9,44,SPAIN,2023-03-05
4,498c0722bf,Real Betis,24,12,5,7,32,25,7,41,SPAIN,2023-03-05
...,...,...,...,...,...,...,...,...,...,...,...,...
15,5301f61042,Deportes Tolima,6,1,3,2,7,9,-2,6,COLOMBIA,2023-03-05
16,c8b4ab6fff,Independiente Santa Fe,6,1,3,2,6,8,-2,6,COLOMBIA,2023-03-05
17,8cd73c5d68,Deportivo Cali,6,1,3,2,4,6,-2,6,COLOMBIA,2023-03-05
18,46bfec9f1d,Alianza Petrolera,5,1,2,2,3,4,-1,5,COLOMBIA,2023-03-05


### Create snowflake table and stage

In [92]:
columnName = list(df_leagues.columns.values)

In [93]:
columnName

['ID',
 'FOOTBALL_TEAM',
 'J',
 'G',
 'E',
 'P',
 'GF',
 'GC',
 'DIF',
 'PTS',
 'COUNTRY',
 'CREATED_TIME']

In [94]:
def getColumnDtypes(dataTypes):
    
    dataList = []
    for x in dataTypes:
        if(x == 'int64'):
            dataList.append('integer')
        elif (x == 'float64'):
            dataList.append('float')
        elif (x == 'bool'):
            dataList.append('boolean')
        else:
            dataList.append('varchar (40)')
    return dataList

columnDataType = getColumnDtypes(df_leagues.dtypes)


In [95]:
columnDataType

['varchar (40)',
 'varchar (40)',
 'integer',
 'integer',
 'integer',
 'integer',
 'integer',
 'integer',
 'integer',
 'integer',
 'varchar (40)',
 'varchar (40)']

In [96]:
createTableStatement = 'create or replace table alexproject.public.football_leagues ('
for i in range(len(columnName)):
    if columnName[i] == 'ID':
        createTableStatement = createTableStatement + '\n' + columnName[i] + ' ' + columnDataType[i] + ' NOT NULL' + ','
    else: 
        createTableStatement = createTableStatement + '\n' + columnName[i] + ' ' + columnDataType[i] + ','
createTableStatement = createTableStatement[:-1] + ' );'

In [97]:
print(createTableStatement)

create or replace table alexproject.public.football_leagues (
ID varchar (40) NOT NULL,
FOOTBALL_TEAM varchar (40),
J integer,
G integer,
E integer,
P integer,
GF integer,
GC integer,
DIF integer,
PTS integer,
COUNTRY varchar (40),
CREATED_TIME varchar (40) );


In [100]:
# create stage
read_snow(test_eng,
"""
create stage alexproject.public.demo_stage
""")

,status
0,Stage area DEMO_STAGE successfully created.


In [101]:
# create table
read_snow(test_eng, f"{createTableStatement}")

,status
0,Table FOOTBALL_LEAGUES successfully created.
